# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [41]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse',engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
X = df['message']
Y = df.drop(['message','id','original','genre'],axis=1)

In [ ]:
# Check if there is null value in each column
Y.isnull().any()

In [5]:
# Check if there is a class whose values are all the same (0 or 1)
print(Y.shape)
Y.sum(axis=0)

(26345, 36)


related                   20298.0
request                    4480.0
offer                       119.0
aid_related               10878.0
medical_help               2087.0
medical_products           1314.0
search_and_rescue           724.0
security                    471.0
military                    860.0
child_alone                   0.0
water                      1674.0
food                       2930.0
shelter                    2319.0
clothing                    406.0
money                       604.0
missing_people              299.0
refugees                    876.0
death                      1196.0
other_aid                  3448.0
infrastructure_related     1705.0
transport                  1203.0
buildings                  1335.0
electricity                 534.0
tools                       159.0
hospitals                   283.0
shops                       120.0
aid_centers                 309.0
other_infrastructure       1151.0
weather_related            7302.0
floods        

In [45]:
# get columns names
class_names = Y.columns.values

In [4]:
# 有一列都是0会导致报错（一个class里需要至少两种值）
# There should be at least 2 kinds of value in a column
Y = Y.fillna(0).drop('child_alone',axis=1)

In [ ]:
# 这种方式不行。原因暂时未知.
# For unknown reason, these method doesn't meet the requirements (for multiOutputClassifier)
Y = np.array(Y,dtype=int)
Y

In [36]:
from sklearn.preprocessing import OneHotEncoder

# 指定 “sparse = False”, 否则输出的是稀疏的存储格式，即索引加值的形式
# Specify "sparse = False", otherwise the output is in sparse storage format, i.e. index plus value
onehot_encoder = OneHotEncoder(sparse = False)
Y = onehot_encoder.fit_transform(Y)
Y

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.]])

### 2. Write a tokenization function to process your text data

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meiji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\meiji\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\meiji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def tokenize(text):
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    
    # Normalize
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    new_text = " ".join(tokens)
    return new_text

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer


pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(X,Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from scipy.stats import loguniform

params = {
            'clf__estimator__n_estimators' : [50,100,200],
            'clf__estimator__min_samples_split': [2, 3, 4],
            'clf__estimator__criterion': ['entropy', 'gini']
        }

cv = RandomizedSearchCV(pipeline,
                        param_distributions=params,
                        n_jobs=-1,cv=3)

cv.fit(X_train, y_train)

C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.18080561        nan 0.1711423  0.15796969 0.18029702
 0.16997254 0.15537585 0.1692605  0.1545621 ]
  category=UserWarning


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('vect',
                                              CountVectorizer(tokenizer=<function tokenize at 0x000001B23A5E75E8>)),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf',
                                              MultiOutputClassifier(estimator=RandomForestClassifier()))]),
                   n_jobs=-1,
                   param_distributions={'clf__estimator__criterion': ['entropy',
                                                                      'gini'],
                                        'clf__estimator__min_samples_split': [2,
                                                                              3,
                                                                              4],
                                        'clf__estimator__n_estimators': [50,
                                                        

In [ ]:
import joblib

# joblib.dump(cv,'classifier.pkl')
cv = joblib.load('classifier.pkl')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# Besides "classification_report", Hamming score is also a way to estimate performance
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [21]:
pred = cv.predict(X_test)
hamming_score(y_test,pred)

0.9662108976276346

In [24]:
# use the sklearn in-build hamming loss function
from sklearn.metrics import hamming_loss

hamming_loss(y_test,pred)

0.018741377161350773

In [51]:
# use classification report
from sklearn.metrics import classification_report,accuracy_score

n_class = len(class_names)
avg = 0
for i in range(n_class):
    accuracy = accuracy_score(y_test[:,i], pred[:,i])
    avg += accuracy
    print(class_names[i])
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test[:,i], pred[:,i]))
    print("\n\n")

avg /= n_class
print(f"Average of accuracy:{avg}")

related
Accuracy: 0.9453768690875801
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.97      5042
         1.0       0.96      0.80      0.87      1512

    accuracy                           0.95      6554
   macro avg       0.95      0.89      0.92      6554
weighted avg       0.95      0.95      0.94      6554




request
Accuracy: 0.9436985047299359
              precision    recall  f1-score   support

         0.0       0.96      0.80      0.87      1568
         1.0       0.94      0.99      0.96      4986

    accuracy                           0.94      6554
   macro avg       0.95      0.90      0.92      6554
weighted avg       0.94      0.94      0.94      6554




offer
Accuracy: 0.9971010070186146
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6498
         1.0       1.00      0.66      0.80        56

    accuracy                           1.00      6554
   macro avg 

              precision    recall  f1-score   support

         0.0       1.00      0.73      0.85       603
         1.0       0.97      1.00      0.99      5951

    accuracy                           0.98      6554
   macro avg       0.99      0.87      0.92      6554
weighted avg       0.98      0.98      0.97      6554




hospitals
Accuracy: 0.9754348489472078
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      5951
         1.0       1.00      0.73      0.85       603

    accuracy                           0.98      6554
   macro avg       0.99      0.87      0.92      6554
weighted avg       0.98      0.98      0.97      6554




shops
Accuracy: 0.9957277998169057
              precision    recall  f1-score   support

         0.0       1.00      0.74      0.85       108
         1.0       1.00      1.00      1.00      6446

    accuracy                           1.00      6554
   macro avg       1.00      0.87      0.92     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.